# Fully connected neural network with Tensorflow for MNIST

## Introduction

Tensorflow is is a symbolic math library and one of the widely used libraries for implementing Machine learning/other algorithms involving large number of mathematical operations. Tensorflow was developed by Google and it’s open source now. It is used for both research and production at Google e.g. for implementing Machine learning in almost all applications 
- Google photos 
- Google voice search 

In this notebook we are going to build a fully connected neural network with Tensorflow

## Requirements

### Imports

In [1]:
import tensorflow as tf
from deep_teaching_commons.data.fundamentals.mnist import Mnist
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

### Loading dataset

The MNIST dataset is a classic Machine Learning dataset you can get it and more information about it from the website of Yann Lecun. MNIST contains handwrittin digits and is split into a tranings set of 60000 examples and a test set of 10000 examples. We use the ```deep_teaching_commons``` package to load the MNIST dataset in a convenient way.

In [2]:
train_images, train_labels, test_images, test_labels = Mnist().get_all_data(one_hot_enc=True, flatten=False)
train_images, test_images = train_images.reshape(60000, 28, 28, 1), test_images.reshape(10000,28,28,1)
print('train shapes:', train_images.shape, train_labels.shape)
print('test shapes:', test_images.shape, test_labels.shape)

auto download is active, attempting download
mnist data directory already exists, download aborted
train shapes: (60000, 28, 28, 1) (60000, 10)
test shapes: (10000, 28, 28, 1) (10000, 10)


### Placeholders

So far we have used numpy arrays to manage our data, but in order to build a model in tensorflow we need another structure, the placeholder. A placeholder is simply a variable that we will assign data to at a later date. It allows us to create our operations and build our computation graph, without needing the data. In TensorFlow terminology, we then feed data into the graph through these placeholders.

In [3]:
# input X: 28x28 grayscale images, the first dimension (None) will index the images in the mini-batch
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
# correct answers will go here
Y = tf.placeholder(tf.float32, [None, 10])

## Fully connected neural network for MNIST


### Initializing the weights
By initializing the weights of our neural network (the learnable parameter), we already define how our network is going to look like. We decided to use a neural network with 3 layer with a ReLU and Dropout function on top of each layer. 

In [4]:
# our neural network architecture:
#
#    · · · · · · ·           (input data, flattened pixels)               X [batch, 784]   # 784 = 28*28
#     \x/x\x/x\x/           -- fully connected layer (ReLU + Droput)      W1 [784, 256]    B3[256]
#      · · · · ·                                                          Y1 [batch, 256]
#       \x/x\x/             -- fully connected layer (ReLU + Droput)      W2 [256, 128]    B4[128]
#        · · ·                                                            Y2 [batch, 128]
#         \x/               -- fully connected layer (softmax)            W3 [128, 10]        B5[10]
#          ·                                                              Y3 [batch, 10]

W1 = tf.Variable(tf.truncated_normal([784, 256], stddev=0.1))  # 784 = 28 * 28
B1 = tf.Variable(tf.zeros([256]))
W2 = tf.Variable(tf.truncated_normal([256, 128], stddev=0.1))
B2 = tf.Variable(tf.zeros([128]))
W3 = tf.Variable(tf.truncated_normal([128, 10], stddev=0.1))
B3 = tf.Variable(tf.zeros([10]))

### Dropout

Dropout is a regularization technique which tries to prevent overfitting. Overfitting means that our network can't perform very well on images it haven't seen before which is obviously really bad.

In [5]:
# Probability of keeping a node during dropout = 1.0 at test time (no dropout) and 0.75 at training time
pkeep = tf.placeholder(tf.float32)

### Building the network

We have, as described above, a 3 layer fully connected neural network with ReLU and Dropout on top of each layer.

In [6]:
flatten = tf.reshape(X, [-1, 784])
hidden1 = tf.nn.relu(tf.matmul(flatten, W1) + B1)
dropout1 = tf.nn.dropout(hidden1, pkeep)
hidden2 = tf.nn.relu(tf.matmul(hidden1, W2) + B2)
dropout2 = tf.nn.dropout(hidden2, pkeep)
output = tf.nn.relu(tf.matmul(hidden2, W3) + B3)

In our **Use Case**, we need a kind of prediction layer on top of our output layer. We use a, so called, Softmax layer or the prediction which we put on top of the output layer. 

In [7]:
prediction = tf.nn.softmax(output)

### Cross Entropy Loss function
In general, the loss functions tells us how "good" or how "bad" our neural network is. This function is then minimized by the neural network so that the neural network gives us the best performance based on the defined loss function. For this purpose we are going to use the cross entropy loss function which is used very heavily in neural networks and seems to work very well.

**Note:** TensorFlow provides the ```softmax_cross_entropy_with_logits``` function to avoid numerical stability problems with log(0) which is NaN

In [8]:
# cross-entropy loss function (= -sum(Y_i * log(Yi)) ), normalised for batches of 100  images
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=Y)
cross_entropy = tf.reduce_mean(cross_entropy)*10

### Optimizer
We are going to use the gradient descent method **Adam** to minimize our loss function. We also add a learning rate with an exponential decay. In our setting we start at a learning rate of $0.003$ and exponentially reduce it to $0.00001$.

In [9]:
# step for variable learning rate
step = tf.placeholder(tf.int32)

# the learning rate is: # 0.0001 + 0.003 * (1/e)^(step/2000)
learning_rate = 0.0001 +  tf.train.exponential_decay(0.003, step, 2000, 1/np.exp(1))
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.minimize(cross_entropy)

### Training the network

We define a ```accuracy``` so that we can see whether our network actually improves while training

In [10]:
correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

0.0928


**Hyperparameter**

In [ ]:
epochs = 100
batch_size = 256

**Main**

In [ ]:
loss_history = []
for e in range(epochs):
    for batch_i in tqdm(range(0, train_images.shape[0], batch_size)):
        data, label = train_images[batch_i:batch_i + batch_size], train_labels[batch_i:batch_i + batch_size]

        # run the computational graph and calculate loss + training step
        with tf.Session() as sess:
            init = tf.global_variables_initializer()
            sess.run(init)
            # optimizer will not return something which is why we store it into a variable called empty
            loss, empty = sess.run([cross_entropy, train_step], feed_dict={X: data, Y: label,  pkeep: 0.75, step: e})
        # append to loss history
        loss_history.append(loss)

    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        train_acc = sess.run(accuracy, feed_dict={X:train_images, Y: train_labels,  pkeep: 1})
        test_acc = sess.run(accuracy, feed_dict={X:test_images, Y: test_labels,  pkeep: 1})
    print('epoch:', e, 'loss:', loss)
    print('test accuracy', test_acc, 'train accuracy', train_acc)

  0%|          | 1/235 [00:00<00:23,  9.79it/s]

epoch: 0 loss: 1332.14
test accuracy 0.098 train accuracy 0.099416666


  0%|          | 1/235 [00:00<00:23,  9.81it/s]

epoch: 1 loss: 1486.891
test accuracy 0.0796 train accuracy 0.0735


  0%|          | 1/235 [00:00<00:27,  8.61it/s]

...

  0%|          | 0/235 [00:00<?, ?it/s]

epoch: 34 loss: 1130.6174
test accuracy 0.1348 train accuracy 0.13745


  0%|          | 0/235 [00:00<?, ?it/s]

epoch: 35 loss: 1818.8826
test accuracy 0.1057 train accuracy 0.10785


 38%|███▊      | 89/235 [01:44<02:51,  1.17s/it]

### Evaluate model
Let us look at the optimization results. Final loss tells us how far we could reduce costs during traning process. Further we can use the first loss value as a sanity check and validate our implementation of the loss function works as intended. Recall loss value after first iteration should be $ log\:c$ with $c$ being number of classes. To visulize the whole tranings process we can plot losss values from each iteration as a loss curve. 

In [ ]:
# check loss after last and first iteration
print('last iteration loss:',loss_history[-1])
print('first iteration loss:',loss_history[0])
# Plot a loss curve
plt.plot(loss_history)
plt.ylabel('loss')
plt.xlabel('iterations')

Evaluation above gave us some inside about the optimization process but did not quantified our final model. One possibility is to calculate model accuracy.

In [ ]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    acc = sess.run(accuracy, feed_dict={X:test_images, Y: test_labels,  pkeep: 1})

print(acc)